<p style="font-size:xx-large">Table of contents</p>

1. [Setup](#setup)
    1. [Imports](#imports)
    2. [Wildfire Information](#wildfire-info)
    3. [File Constants](#file-constants)
2. [Data Collection](#data-collection)
3. [Image Processing](#image-processing)
    1. [Wildifre Area](#wildfire-area)
    2. [Morphology](#morphology)
4. [Land Cover Classification](#land-cover-classification)
    1. [Land Cover Datasets](#land-cover-datasets)
    2. [Interactive Map](#interactive-map)
5. [Wind Data](#wind-data)
    1. [Create Wind Map](#wind-map)

<a name="setup"></a>
# Setup

<a name="imports"></a>
## Imports

In [ ]:
import datetime as dt
import json
import os

import ee
import matplotlib.pyplot as plt
import numpy as np

import utils.user_inputs as ui

from sentinelsat import SentinelAPI

In [ ]:
# If you have never authenticated, this will raise an exception
# and prompt you to authenticate. You only need to do this once.
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

Input the name of the location where the wildfire occurred. Examples are:
* Istres
* Montguers
* etc.

In [ ]:
# Name of the place where the fire is located
FIRE_NAME = ui.get_fire_name()

# Folder where the JP2 images will be stored
PATH = f'data/{FIRE_NAME}/'
# Path to the folders where the TIFF, PNG, and GIF files will be stored
OUTPUT_FOLDER = f"output/{FIRE_NAME}/"
OUTPUT_MAPS = OUTPUT_FOLDER + "maps/"
OUTPUT_PLOTS = OUTPUT_FOLDER + "plots/"

<a name="wildfire-info"></a>
## Wildfire information

In [ ]:
with open(f"data/info_fires/info_{FIRE_NAME}.json") as f:
    fire_info = json.load(f)

# Date of the fire
WILDFIRE_DATE = dt.datetime.strptime(fire_info["wildfire_date"], "%Y-%m-%d")
# Coordinates of the fire
LATITUDE, LONGITUDE = fire_info["latitude"], fire_info["longitude"]
# Actual area in hectares that burned. We retrieved the area on the news
TRUE_AREA = fire_info["true_area"]

In [ ]:
# Path to the GeoJSON file
GEOJSON_PATH = f"data/geojson_files/{FIRE_NAME}.geojson"
# Path to the JSON file where the Sentinel API credentials are stored
CREDENTIALS_PATH = "secrets/sentinel_api_credentials.json"

<a name="file-constants"></a>
## File constants

* <u>Observation interval</u>:
<p>The observation interval is somewhat arbitrary, but longer ranges allow us to retrieve more images, since they are taken every 5 days. Furthermore, the burned area is visible for a few weeks, even months, after the fire started.</p>


* <u>Resolution</u>:
<p>As of right now, we only use the <b>NDVI</b> of an image for the processing steps. The necessary bands, <code>B08</code> (near-infrared) and <code>B04</code> (red) are available at all resolutions (10, 20, and 60 m) but for better results we use 10.
Other indexes, such as <i>SWIR</i>, need resolutions 20 or 60 but are not yet implemented.</p>


* <u>Cloud threshold</u>:
<p>We only retrieve the images below this threshold, because otherwise they are mostly no-data images or yield no valuable results.</p>


* <u>Samples</u>:
<p>After retrieving the land cover data, we use sample coordinates to create the pie charts.</p>

In [ ]:
# Number of days both before and after the fire to get images
OBSERVATION_INTERVAL = 15
# Resolution of the images (10 m, 20 m, or 60 m)
RESOLUTION = 10
# Threshold for the cloud cover (between 0 and 100)
CLOUD_THRESHOLD = 40
# Seed for random number generator (for reproductibility)
SEED = 42
# Number of coordinates to use for the pie charts
SAMPLES = np.arange(50, 1001, 50)

<a name="data-collection"></a>
# Data Collection

In [ ]:
from utils.data_collection import get_before_after_images, check_downloaded_data

In [ ]:
with open(CREDENTIALS_PATH, 'r') as infile:
    credentials = json.load(infile)

api = SentinelAPI(
    credentials["username"],
    credentials["password"]
)

In [ ]:
for path in [PATH, OUTPUT_FOLDER, OUTPUT_MAPS, OUTPUT_PLOTS]:
    if not os.path.exists(path):
        os.makedirs(path)

This is the main function that retrieves and downloads the relevant images. Many functions are called, available in [data_collection.py](utils/data_collection.py). The main idea is that we retrieve the images with the most information: less cloud coverage, less water presence, and larger in size (since no-data images are smaller).

You may get the error __Product ... is not online. Triggered retrieval from Long Term Archive__, specially with older images. Unfortunately, the only solution we have found is to wait for 15-30 minutes (maybe more) and then try again.

Another error, __NullPointerException__ may occur, but the solution is the same: try again after a few minutes.

If you get the error __Index .. is out of bounds for axis 0 with size ..__, it means that a suitable image was not found. To solve the problem, you can try extending the observation interval by a few days, or lower the minimum size required for the images (it is set at 980 MB) inside the data collection file.

In [ ]:
if not check_downloaded_data(PATH, OUTPUT_FOLDER, FIRE_NAME):
    try:
        get_before_after_images(
            api=api,
            wildfire_date=WILDFIRE_DATE,
            geojson_path=GEOJSON_PATH,
            observation_interval=OBSERVATION_INTERVAL,
            path=PATH,
            fire_name=FIRE_NAME,
            output_folder=OUTPUT_FOLDER,
            resolution=RESOLUTION,
            cloud_threshold=CLOUD_THRESHOLD
        )
    except Exception as e:
        print(e)
        exit()

<a name="image-processing"></a>
# Image Processing

The first step here is to retrieve the location of the fire inside the produced NDVI image. Instead of doing this manually, we can transform the coordinates from the JSON info file into pixel values. Then, we plot the pixels inside the image.

In [ ]:
import utils.image_processing as ip

In [ ]:
# ip.plot_downloaded_images(FIRE_NAME, OUTPUT_FOLDER, save=True)

In [ ]:
# The necessary information is stored in the following folder:
img_folder = PATH + os.listdir(PATH)[1] + '/'
print(img_folder)

pixel_column, pixel_row = ip.get_fire_pixels(
    img_folder, LATITUDE, LONGITUDE
)

In [ ]:
diff = ip.get_ndvi_difference(
    OUTPUT_FOLDER, FIRE_NAME, save_diff=False
)
ax = ip.imshow(diff, figsize=(10, 10), title='NDVI Difference')
plt.savefig(f'{OUTPUT_PLOTS}ndvi_difference.png', dpi=200)
plt.show()

In [ ]:
ax = ip.imshow(diff, figsize=(10, 10), title='NDVI Difference with Wildfire Location')
ip.plot_location(ax, pixel_column, pixel_row)
plt.savefig(f'{OUTPUT_PLOTS}ndvi_difference_w_fire.png', dpi=200)
plt.show()

Next, we ask the user to give pixel values to add lines to the image to zoom-in on the area of interest. We have explored methods to do this automatically but we have not been successful for now.

In [ ]:
print(f'The fire is located at pixels ({pixel_column}, {pixel_row}).\n')

In [ ]:
fire, hline_1, vline_1 = ip.retrieve_fire_area(
    diff, pixel_column, pixel_row,
    figsize=(10, 10), title='Fire Area'
)
plt.savefig(f'{OUTPUT_PLOTS}fire_area.png', dpi=200)
plt.show()

<a name="wildifre-area"></a>
## Wildfire area

Here we calculate the area that burned thanks to the difference in NDVI and we plot the results, along with the true value. This helps us to validate our functions.

In order to obtain the optimal threshold value, we compute the area for different values of thresholds and keep the one that gives the best approximation.

In [ ]:
thresholds, areas = ip.get_thresholds_areas(fire, RESOLUTION)

In [ ]:
ip.plot_area_vs_threshold(thresholds, areas, TRUE_AREA)
plt.savefig(f'{OUTPUT_PLOTS}fire_area_thresholds.png', dpi=200)
plt.show()

In [ ]:
threshold = ip.get_threshold(thresholds, areas, TRUE_AREA)
threshold

In [ ]:
tmp = ip.threshold_filter(fire, threshold)
ax = ip.imshow(tmp, figsize=(10, 10), title=f'Thresholded Fire\nwith threshold = {threshold}')
plt.savefig(f'{OUTPUT_PLOTS}thresholded_fire.png', dpi=200)
plt.show()

In [ ]:
print('Calculated area:', round(ip.calculate_area(tmp) * 100, 4), 'ha.')

In [ ]:
print(f'The true area that burned is {TRUE_AREA} hectares.\n')

<a name="morphology"></a>
## Morphology

For the final step in the image processing part, we use mathematical morphology to slighlty improve the quality of the image. More details are available here: [https://scikit-image.org/docs/stable/api/skimage.morphology.html](https://scikit-image.org/docs/stable/api/skimage.morphology.html).

In [ ]:
from skimage.morphology import area_closing

closed = area_closing(tmp, connectivity=2)
ip.plot_comparison(tmp, closed, 'Area Closing')
plt.savefig(f'{OUTPUT_PLOTS}area_closing.png', dpi=200)
plt.show()

In [ ]:
print('Area after morphology:', round(ip.calculate_area(closed) * 100, 4), 'ha.')

You can see a small difference between the images: the bright areas are more "connected", whereas the dark spots are dimmer. This is why the calculated area is a little larger.

Once you're satisfied with the results, execute the next cell.

In [ ]:
fire = closed.copy()
del tmp
del closed

<a name="land-cover-classification"></a>
# Land Cover Classification

<a name="land-cover-datasets"></a>
## Land cover datasets

The next step consists in retrieving information on the type of land that was affected by the fire, such as crops or forests. Multiple datasets are available from Earth Engine's catalog. They use different resolutions and time ranges. The ones we use are:
* [MODIS Land Cover](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MCD12Q1?hl=en) _(2018, 500 m)_
* [ESA World Cover](https://developers.google.com/earth-engine/datasets/catalog/ESA_WorldCover_v100) _(2020, 10 m)_
* [Copernicus Global Land Service](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global) _(2019, 100 m)_
* [Copernicus CORINE Land Cover](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_CORINE_V20_100m) _(2018, 100 m)_. *Note: this dataset only covers Europe, so keep this in mind if you want information about wildfires in other continents.*

In [ ]:
import utils.land_coverage as land_c

In [ ]:
choice = land_c.get_choice()

This is the selected land cover dataframe:

In [ ]:
lc_dataframe = land_c.get_land_cover_dataframe(choice)
lc_dataframe

In [ ]:
prob = ui.get_percentage(case='land use')

In [ ]:
rand_image = land_c.create_sample_coordinates(fire, SEED, prob)
land_c.plot_sampled_coordinates(rand_image, prob, figsize=(10, 10), cmap='hot')
plt.savefig(f'{OUTPUT_PLOTS}sampled_coordinates.png', dpi=200)
plt.show()

Conversely, we can transform pixel values from the image into coordinates.

In [ ]:
coordinates = land_c.get_coordinates_from_pixels(
    rand_image, hline_1, vline_1, img_folder, FIRE_NAME
)
coordinates

In [ ]:
output_folder = f"output/{FIRE_NAME}/pie_charts/"
exists = os.path.exists(output_folder)
if not exists:
    os.makedirs(output_folder)
is_empty = not any(os.scandir(output_folder))

After asking the user for a land cover dataset, we create a GIF file from the pie charts, to see the evolution of land cover information we obtain.

In [ ]:
if exists and is_empty or not exists:
    land_c.create_plots(
        samples=SAMPLES,
        coordinates=coordinates,
        choice=choice,
        seed=SEED,
        fire_name=FIRE_NAME,
        out_folder=output_folder,
        save_fig=True
    )

In [ ]:
land_c.make_pie_chart_gif(
    fire_name=FIRE_NAME,
    file_path=output_folder,
    save_all=True,
    duration=500,
    loop=0
)

You can open the GIF using your default program:

In [ ]:
land_c.open_gif(FIRE_NAME, output_folder)

Or, alternatively, you can display the GIF directly in the notebook:

In [ ]:
import base64
from IPython import display

def show_gif(fname):
    with open(fname, 'rb') as fd:
        b64 = base64.b64encode(fd.read()).decode('ascii')
    return display.HTML(f'<img src="data:image/gif;base64,{b64}" />')

show_gif(f'output/{FIRE_NAME}/pie_charts/{FIRE_NAME}.gif')

<a name="interactive-map"></a>
## Create interactive map

Finally, we create an interactive map using `geemap` to visualize the coordinates of the fire on a map, also adding the land cover layer.

In [ ]:
from utils.plot_map import create_map, save_map, open_map

In [ ]:
prob = ui.get_percentage(case='map')

In [ ]:
fire_map = create_map(
    FIRE_NAME, prob, choice,
    seed=SEED,
    zoom=5,
    cluster=True,
    minimap=False
)

The main advantage of notebooks is that we can also display the map in a cell, instead of in the browser:

In [ ]:
fire_map

**Please note** that if you choose the option `cluster=False`, the markers will not appear in the saved file. They only appear when you add a cluster to the map. Moreover, the legend of land cover types will not appear either. Refer to the `get_legend` function inside the [utils.plot_map](utils/plot_map.py) file for more information.

In [ ]:
save_map(fire_map, FIRE_NAME, OUTPUT_MAPS, wind=False)
open_map(OUTPUT_MAPS, wind=False)

<a name="wind-data"></a>
# Wind Data

In [ ]:
import utils.wind_data as wind
from ipyleaflet import basemaps, basemap_to_tiles

In [ ]:
year = WILDFIRE_DATE.strftime('%Y')
month = WILDFIRE_DATE.strftime('%m')
day = WILDFIRE_DATE.strftime('%d')
hours = ['12:00']
center = (LATITUDE, LONGITUDE)

In [ ]:
output_file = wind.retrieve_wind_data(FIRE_NAME, year, month, day, hours)
print('Output file:', output_file)

In [ ]:
ds = wind.open_nc_data(output_file)
print(ds)

In [ ]:
ds = wind.reshape_data(ds)

<a name="wind-map"></a>
## Create wind map

In [ ]:
wind_map = wind.create_map(
    ds, center, choice,
    zoom=5,
    # basemap=basemaps.CartoDB.DarkMatter,
    basemap=basemaps.Esri.WorldImagery
)

In [ ]:
wind_map

You can execute the next cell to add a layer to the map, which will automatically update. More basemaps are available here: https://ipyleaflet.readthedocs.io/en/latest/api_reference/basemaps.html.

In [ ]:
# add a basemap as layer
# m.add_layer(basemap_to_tiles(basemaps.Esri.WorldImagery))
wind_map.add_layer(basemap_to_tiles(basemaps.CartoDB.DarkMatter))

In [ ]:
save_map(wind_map, FIRE_NAME, OUTPUT_MAPS, wind=True)
open_map(OUTPUT_MAPS, wind=True)